# This notebook illsutrates how to evaluate the performance of runs

The main function used is evaluate_project from evaluate.py. To use the function 2 main thigs need to be defined:
- which dataset directories to use:
    - `data_dir` = for whole test set 
    - `idr_data_dir_pattern` = for IDR test set
- which models to evaluate. To supply these **ONE** of 3 options need to be done:
    - `run_dir_list`  = list of pre-defined runs 
    - `project_dir` = a directory that contains subdirectories of models that will be evaluated
    - `wandb_project_name` = project completed in WandB. If using this option must also provide `wandb_dir` = a directory path where the outputs of the project are saved

Optionally output path `output_dir`, filename `output_prefix` can be defined and `batch_size` can be defined. Set batch size to smaller values for bigger models (at the cost of longer duration)

The outputs are:
- csv of the evaluation results wih 4 rows per target of a model corresponding to whole or IDR, scaled or raw results (so the csv is made up of *N(models) * N(targets) * 4 rows*
- csv of the scaling factors 

In [13]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../scripts')
import evaluate
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
%%time
# use list of run directories
evaluate.evaluate_project(data_dir='/home/shush/profile/QuantPred/datasets/chr8/complete/random_chop/i_2048_w_1/',
                     idr_data_dir_pattern='/mnt/1a18a49e-9a31-4dbf-accd-3fb8abbfab2d/shush/15_IDR_test_sets_6K/cell_line_*/i_6144_w_1/',
                          run_dir_list=['/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/wandb/wandb_amber/run-20210804_161352-v3cllxs2/'],
                         batch_size=512)

USING PREDEFINED LIST OF RUNS
/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/wandb/wandb_amber/run-20210804_161352-v3cllxs2/
CPU times: user 5min 52s, sys: 2min 32s, total: 8min 24s
Wall time: 4min 5s


In [19]:
evaluation_results = pd.read_csv('output/evaluation_results.csv') # get results from default output path
evaluation_results.head(2)

,index,mse,js_per_seq,js_conc,poiss,pr_corr,sp_corr,targets,pred type,eval type,...,data_dir,filtN_1,kern_3,l_rate,loss_fn,lr_patience,model_fn,num_epochs,window_size,run_dir
0,0,0.583636,0.265415,0.296701,0.756931,0.567362,0.430623,GM21381,raw,whole,...,./datasets/complete/random_chop/i_3072_w_1,128,25,0.001,poisson,6,bpnet,30,2048,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...
1,1,0.579878,0.250584,0.305470,0.793883,0.658281,0.667384,GM23338,raw,whole,...,./datasets/complete/random_chop/i_3072_w_1,128,25,0.001,poisson,6,bpnet,30,2048,/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shus...


In [31]:
eval_type = 'whole'
pred_type = 'raw'

raw_whole_results = evaluation_results[(evaluation_results['pred type']==pred_type)&
                                      (evaluation_results['eval type']==eval_type)]
print('Average Pearson\'s r per run')
raw_whole_results.groupby('run_dir').mean()

Average Pearson's r per run


,index,mse,js_per_seq,js_conc,poiss,pr_corr,sp_corr,bin_size,filtN_1,kern_3,l_rate,lr_patience,num_epochs,window_size
run_dir,,,,,,,,,,,,,,
/mnt/31dac31c-c4e2-4704-97bd-0788af37c5eb/shush/wandb/wandb_amber/run-20210804_161352-v3cllxs2/,7,0.659417,0.317675,0.352277,0.803742,0.595373,0.499104,32,128,25,0.001,6,30,2048


In [30]:
# per cell line pearson r results
raw_whole_results.sort_values('pr_corr')[['targets', 'pr_corr']]

,targets,pr_corr
4,HG03575,0.438756
6,DND-41,0.527011
13,PC-3,0.548945
0,GM21381,0.567362
10,IMR-90,0.578082
3,RWPE2,0.584969
11,NCI-H929,0.596047
14,MCF,0.600011
7,GM12878,0.606206
12,Panc1,0.609240
